In [ ]:
import pandas as pd
import requests

# API Key (Replace with your actual FRED API key)
FRED_API_KEY = "c0c8c88711d864a221baf691fb4680ff"

# Macroeconomic indicators to fetch
FRED_INDICATORS = {
    "GDP": "GDP",  # US Gross Domestic Product
    "Inflation": "CPIAUCSL",  # Consumer Price Index
    "Interest Rate": "DFF",  # Federal Funds Rate
}

# Function to fetch macroeconomic data from FRED
def fetch_fred_data(indicator_code):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={indicator_code}&api_key={FRED_API_KEY}&file_type=json"
    response = requests.get(url).json()

    if "observations" not in response:
        print(f"⚠️ Error fetching {indicator_code}: {response}")
        return None

    # Convert response to DataFrame
    data = pd.DataFrame(response["observations"])
    data["date"] = pd.to_datetime(data["date"])
    data.set_index("date", inplace=True)

    # Ensure 'value' column is numeric, replacing non-numeric values
    data["value"] = pd.to_numeric(data["value"], errors="coerce")  # Convert to float, replace errors with NaN

    return data.dropna()  # Remove rows with NaN values

# Fetch all macroeconomic indicators
macro_data = {name: fetch_fred_data(code) for name, code in FRED_INDICATORS.items()}

# Function to calculate required averages
def calculate_and_save(df, name, filename):
    if df is None or df.empty:
        print(f"⚠️ No data available for {name}.")
        return

    # Print available years for debugging
    available_years = df.index.year.unique()
    print(f"✅ Available years for {name}: {available_years}")

    # Define the most recent year available
    last_available_year = df.index.year.max()

    # Previous 2 years' average
    previous_two_years = [last_available_year - 1, last_available_year - 2]
    prev_2_years_avg = df[df.index.year.isin(previous_two_years)]["value"].mean()

    # Current year (or last available year) average
    current_year_data = df[df.index.year == last_available_year]

    if current_year_data.empty:
        print(f"⚠️ No data available for {name} in {last_available_year}. Using previous year instead.")
        last_available_year -= 1
        current_year_data = df[df.index.year == last_available_year]

    # Calculate overall average for the current year
    current_year_avg = current_year_data["value"].mean()

    # Create a DataFrame to store results
    results_df = pd.DataFrame({
        "Metric": ["Previous 2 Years Avg", "Current Year Avg"],
        "Value": [prev_2_years_avg, current_year_avg]  # Store only one "Current Year Avg"
    })

    # Save results to CSV
    results_df.to_csv(filename, index=False)
    print(f"✅ {name} data saved to {filename}")

# Calculate and save for GDP, Inflation, and Interest Rate
calculate_and_save(macro_data["GDP"], "GDP", "gdp.csv")
calculate_and_save(macro_data["Inflation"], "Inflation", "inflation.csv")
calculate_and_save(macro_data["Interest Rate"], "Interest Rate", "interest_rate.csv")
